# Imports and Constants

In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

import tensorflow as tf
from tensorflow import keras

from src.supervised.train import train_model
from src.supervised.evaluate import evaluate_model
from src.supervised.build.build import full_model, example_config
from src import constants

ckpt_dir = "ckpt/critic"
model_type = "critic"

# Models

In [9]:
# conv output 256, layer 4
# dense layer 0

board_input = keras.Input(shape=(64, 13), name="board")
misc_input = keras.Input(shape=(6), name="misc")

conv_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 13), input_shape=(64, 13)),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.Flatten(),
    keras.layers.Dense(256)
], name="convolutional_model")
conv_output = conv_model(board_input)

dense_inputs = keras.layers.concatenate([conv_output, misc_input])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.k(262),
    keras.layers.Dense(1, activation="sigmoid")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = tf.keras.models.Model([board_input, misc_input], dense_output, name="co256_cl4_d0")
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = keras.losses.BinaryCrossentropy(),
    metrics = [keras.metrics.BinaryCrossentropy()],
)
model.summary()

Model: "co256_cl4_d0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 board (InputLayer)             [(None, 64, 13)]     0           []                               
                                                                                                  
 convolutional_model (Sequentia  (None, 256)         1167168     ['board[0][0]']                  
 l)                                                                                               
                                                                                                  
 misc (InputLayer)              [(None, 6)]          0           []                               
                                                                                                  
 concatenate_2 (Concatenate)    (None, 262)          0           ['convolutional_model[

In [10]:
train_model(model, model_type, n_files=1, ckpt_dir=ckpt_dir)

Training on file chess1
   37/14063 [..............................] - ETA: 3:40 - loss: 0.6964 - binary_crossentropy: 0.6964

KeyboardInterrupt: 

In [ ]:
evaluate_model(model, model_type, ckpt_dir=ckpt_dir)